In [1]:
!pip install konlpy
!pip install torchtext

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 3.8MB 36.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
from konlpy.tag import Okt
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준
from torchtext import data  
from konlpy.tag import Mecab
import urllib.request
import pandas as pd

In [26]:

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f5c15830710>)

In [5]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [6]:
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)


In [7]:
tokenizer = Mecab()
# 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

In [8]:
from torchtext.data import TabularDataset
train_data, test_data = TabularDataset.splits(
        path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [140]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 150000
테스트 샘플의 개수 : 50000


In [141]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [152]:
class grumodel(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grumodel, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim*4,self.hidden_dim*2)
        self.mlp2 = nn.Linear(self.hidden_dim*2,self.hidden_dim)
        self.mlp3 = nn.Linear(self.hidden_dim,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)
        x = self.dropout(x)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp2(x)
        x = self.relu(x)
        x = self.mlp3(x)
        x = self.sigmoid(x).squeeze()
        return x


In [175]:
def train(model, optimizer, loss_function,train_iter,DEVICE):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()

        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        loss.backward()
        optimizer.step()

In [180]:
def evaluate(model, val_iter, loss_function, DEVICE):
    """evaluate model"""
    model.eval()
    total_loss = 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        total_loss += loss.item()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    return avg_loss

In [160]:
BATCH_SIZE = 100
'''
train_iter, val_iter = data.BucketIterator.splits(
        (train_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)
'''
from torchtext.data import Iterator
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
val_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [156]:
device = torch.device("cuda")
model = grumodel(128,10002,256,1,100,0.3)
model.to(device)
loss = nn.BCELoss()
lr = 0.001

EPOCHS = 10
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [181]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, loss,train_loader,device)
    val_loss = evaluate(model, val_loader, loss, device)

    print("[Epoch: %d] val loss : %1.5f " % (e, val_loss))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        print("Best saved")
        torch.save(model.state_dict(), '/content/gdrive/My Drive/GRUmodel/NLP_esemble_model.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss : 0.00440 
Best saved
[Epoch: 2] val loss : 0.00439 
Best saved
[Epoch: 3] val loss : 0.00458 
[Epoch: 4] val loss : 0.00431 
Best saved
[Epoch: 5] val loss : 0.00427 
Best saved
[Epoch: 6] val loss : 0.00440 
[Epoch: 7] val loss : 0.00439 
[Epoch: 8] val loss : 0.00444 
[Epoch: 9] val loss : 0.00498 
[Epoch: 10] val loss : 0.00457 
